# loan approved or rejected

In [112]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer,SimpleImputer

from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, multilabel_confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

In [113]:
df = pd.read_csv('loan_data.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


# preprocessing

In [114]:
df.drop(['Loan_ID','Gender'],axis=1,inplace=True)

In [115]:
df

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [116]:
df['Married'] = df['Married'].replace({'Yes':1,'No':0})

In [117]:
df['Dependents'].value_counts()

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [118]:
df['Property_Area'].value_counts()

Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64

In [119]:
df.isna().sum()

Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [120]:
df['Education'] = df['Education'].replace({'Not Graduate':0,'Graduate':1})
df['Self_Employed'] = df['Self_Employed'].replace({'No':0,'Yes':1})
df['Property_Area'] = df['Property_Area'].replace({'Urban':0,'Rural':1,'Semiurban':2})

In [121]:
df['Dependents'] = df['Dependents'].replace({0:0,1:0,2:2,'3+':3}).astype(float)

In [123]:
df['Loan_Status'] = df['Loan_Status'].replace({'N':0,'Y':1}).astype(float)

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Married            611 non-null    float64
 1   Dependents         599 non-null    float64
 2   Education          614 non-null    int64  
 3   Self_Employed      582 non-null    float64
 4   ApplicantIncome    614 non-null    int64  
 5   CoapplicantIncome  614 non-null    float64
 6   LoanAmount         592 non-null    float64
 7   Loan_Amount_Term   600 non-null    float64
 8   Credit_History     564 non-null    float64
 9   Property_Area      614 non-null    int64  
 10  Loan_Status        614 non-null    float64
dtypes: float64(8), int64(3)
memory usage: 52.9 KB


In [62]:
df['Dependents'].value_counts()

0.0    345
1.0    102
2.0    101
3.0     51
Name: Dependents, dtype: int64

# simple imputer for features only

In [125]:
s_imputer = SimpleImputer(strategy="most_frequent")
result = s_imputer.fit_transform(df)
new_df = pd.DataFrame(result, columns = df.columns,dtype=int)
new_df

C:\Users\Shruti\AppData\Local\Temp/ipykernel_16460/3034273607.py:3: FutureWarning: In a future version, passing float-dtype values and an integer dtype to DataFrame will retain floating dtype if they cannot be cast losslessly (matching Series behavior). To retain the old behavior, use DataFrame(data).astype(dtype)
  new_df = pd.DataFrame(result, columns = df.columns,dtype=int)


,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,0,1,0,5849,0,120,360,1,0,1
1,1,1,1,0,4583,1508,128,360,1,1,0
2,1,0,1,1,3000,0,66,360,1,0,1
3,1,0,0,0,2583,2358,120,360,1,0,1
4,0,0,1,0,6000,0,141,360,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,1,0,2900,0,71,360,1,1,1
610,1,3,1,0,4106,0,40,180,1,1,1
611,1,1,1,0,8072,240,253,360,1,0,1
612,1,2,1,0,7583,0,187,360,1,0,1


In [126]:
new_df.isna().sum()

Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [127]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Married            614 non-null    int32
 1   Dependents         614 non-null    int32
 2   Education          614 non-null    int32
 3   Self_Employed      614 non-null    int32
 4   ApplicantIncome    614 non-null    int32
 5   CoapplicantIncome  614 non-null    int32
 6   LoanAmount         614 non-null    int32
 7   Loan_Amount_Term   614 non-null    int32
 8   Credit_History     614 non-null    int32
 9   Property_Area      614 non-null    int32
 10  Loan_Status        614 non-null    int32
dtypes: int32(11)
memory usage: 26.5 KB


In [128]:
new_df.columns

Index(['Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [129]:
counter = 0
for i in new_df.columns:
    print(f'{new_df.columns[counter]}:\n{new_df[i].unique()}')
    counter +=1

Married:
[0 1]
Dependents:
[0 1 2 3]
Education:
[1 0]
Self_Employed:
[0 1]
ApplicantIncome:
[ 5849  4583  3000  2583  6000  5417  2333  3036  4006 12841  3200  2500
  3073  1853  1299  4950  3596  3510  4887  2600  7660  5955  3365  3717
  9560  2799  4226  1442  3750  4166  3167  4692  3500 12500  2275  1828
  3667  3748  3600  1800  2400  3941  4695  3410  5649  5821  2645  4000
  1928  3086  4230  4616 11500  2708  2132  3366  8080  3357  3029  2609
  4945  5726 10750  7100  4300  3208  1875  4755  5266  1000  3333  3846
  2395  1378  3988  2366  8566  5695  2958  6250  3273  4133  3620  6782
  2484  1977  4188  1759  4288  4843 13650  4652  3816  3052 11417  7333
  3800  2071  5316  2929  3572  7451  5050 14583  2214  5568 10408  5667
  2137  2957  3692 23803  3865 10513  6080 20166  2014  2718  3459  4895
  3316 14999  4200  5042  6950  2698 11757  2330 14866  1538 10000  4860
  6277  2577  9166  2281  3254 39999  9538  2980  1863  7933  3089  4167
  9323  3707  2439  2237  8000  

### By observing we can say that there is no null value or any special character is present in columns

In [88]:
new_df.columns

Index(['Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [130]:
new_df['Loan_Status'].value_counts()

1    422
0    192
Name: Loan_Status, dtype: int64

In [197]:
new_df.corr()

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Married,1.000000,0.334216,-0.012304,0.004489,0.051708,0.075947,0.146212,-0.100912,0.010938,0.003071,0.091478
Dependents,0.334216,1.000000,-0.055752,0.056798,0.118202,0.030429,0.163017,-0.103864,-0.040160,0.001781,0.010118
Education,-0.012304,-0.055752,1.000000,0.010383,0.140760,0.062290,0.169436,0.073928,0.073658,0.003592,0.085884
Self_Employed,0.004489,0.056798,0.010383,1.000000,0.127180,-0.016100,0.114971,-0.033739,-0.001550,0.021996,-0.003700
ApplicantIncome,0.051708,0.118202,0.140760,0.127180,1.000000,-0.116604,0.564698,-0.046531,-0.018615,-0.007894,-0.004710
CoapplicantIncome,0.075947,0.030429,0.062290,-0.016100,-0.116604,1.000000,0.189723,-0.059382,0.011134,-0.028355,-0.059188
LoanAmount,0.146212,0.163017,0.169436,0.114971,0.564698,0.189723,1.000000,0.037152,-0.000250,0.018104,-0.031808
Loan_Amount_Term,-0.100912,-0.103864,0.073928,-0.033739,-0.046531,-0.059382,0.037152,1.000000,-0.004705,0.086879,-0.022549
Credit_History,0.010938,-0.040160,0.073658,-0.001550,-0.018615,0.011134,-0.000250,-0.004705,1.000000,0.030248,0.540556
Property_Area,0.003071,0.001781,0.003592,0.021996,-0.007894,-0.028355,0.018104,0.086879,0.030248,1.000000,0.103253


In [131]:
new_df.describe()

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.00000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,0.653094,0.744300,0.781759,0.133550,5403.459283,1621.24430,145.465798,342.410423,0.855049,1.050489,0.687296
std,0.476373,1.009623,0.413389,0.340446,6109.041673,2926.24876,84.180967,64.428629,0.352339,0.840875,0.463973
min,0.000000,0.000000,0.000000,0.000000,150.000000,0.00000,9.000000,12.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,2877.500000,0.00000,100.250000,360.000000,1.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,3812.500000,1188.50000,125.000000,360.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,5795.000000,2297.25000,164.750000,360.000000,1.000000,2.000000,1.000000
max,1.000000,3.000000,1.000000,1.000000,81000.000000,41667.00000,700.000000,480.000000,1.000000,2.000000,1.000000


# Train test split

In [132]:
df_train = new_df.copy()
x = df_train.drop('Loan_Status', axis = 1)
y = df_train['Loan_Status']

In [133]:
std_scaler = StandardScaler()
std_scaler.fit(x)  # Saving mean and std

StandardScaler()

In [134]:
standard_array = std_scaler.transform(x)
x_df_std = pd.DataFrame(standard_array, columns = x.columns)
x_df_std

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,-1.372089,-0.737806,0.528362,-0.392601,0.072991,-0.554487,-0.302759,0.273231,0.411733,-1.250298
1,0.728816,0.253470,0.528362,-0.392601,-0.134412,-0.038731,-0.207648,0.273231,0.411733,-0.060092
2,0.728816,-0.737806,0.528362,2.547117,-0.393747,-0.554487,-0.944757,0.273231,0.411733,-1.250298
3,0.728816,-0.737806,-1.892641,-0.392601,-0.462062,0.251980,-0.302759,0.273231,0.411733,-1.250298
4,-1.372089,-0.737806,0.528362,-0.392601,0.097728,-0.554487,-0.053093,0.273231,0.411733,-1.250298
...,...,...,...,...,...,...,...,...,...,...
609,-1.372089,-0.737806,0.528362,-0.392601,-0.410130,-0.554487,-0.885313,0.273231,0.411733,-0.060092
610,0.728816,2.236021,0.528362,-0.392601,-0.212557,-0.554487,-1.253868,-2.522836,0.411733,-0.060092
611,0.728816,0.253470,0.528362,-0.392601,0.437174,-0.472404,1.278459,0.273231,0.411733,-1.250298
612,0.728816,1.244745,0.528362,-0.392601,0.357064,-0.554487,0.493794,0.273231,0.411733,-1.250298


In [ ]:
Need to use SMOTE for balancing the data

In [136]:
from imblearn.over_sampling import SMOTE

In [137]:
smt = SMOTE()
x_samples, y_samples = smt.fit_resample(x_df_std,y)
y_samples.value_counts()

1    422
0    422
Name: Loan_Status, dtype: int64

In [138]:
x_samples

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,-1.372089,-0.737806,0.528362,-0.392601,0.072991,-0.554487,-0.302759,0.273231,0.411733,-1.250298
1,0.728816,0.253470,0.528362,-0.392601,-0.134412,-0.038731,-0.207648,0.273231,0.411733,-0.060092
2,0.728816,-0.737806,0.528362,2.547117,-0.393747,-0.554487,-0.944757,0.273231,0.411733,-1.250298
3,0.728816,-0.737806,-1.892641,-0.392601,-0.462062,0.251980,-0.302759,0.273231,0.411733,-1.250298
4,-1.372089,-0.737806,0.528362,-0.392601,0.097728,-0.554487,-0.053093,0.273231,0.411733,-1.250298
...,...,...,...,...,...,...,...,...,...,...
839,0.728816,-0.305086,0.528362,2.547117,-0.083375,-0.554487,-0.478497,0.273231,-2.428760,0.199688
840,0.728816,1.244745,0.528362,-0.392601,-0.387147,0.035406,-0.216196,0.273231,-2.428760,1.130115
841,0.728816,0.481939,-1.892641,-0.392601,-0.216067,0.953647,-0.113421,0.273231,0.411733,-0.060092
842,0.728816,-0.737806,-1.892641,-0.392601,-0.528003,0.329391,-0.015068,0.273231,-2.428760,-1.250298


# train test split

In [139]:
x_train, x_test, y_train, y_test = train_test_split(x_samples, y_samples, test_size = 0.45, random_state=11, stratify=y_samples)
x_train

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
495,0.728816,0.253470,0.528362,-0.392601,-0.340340,-0.242229,-0.481092,-4.014071,0.411733,1.130115
141,-1.372089,-0.737806,0.528362,-0.392601,0.002218,-0.554487,0.267906,0.273231,0.411733,-1.250298
661,0.469924,0.131316,0.528362,-0.392601,4.218580,-0.554487,1.221036,0.273231,0.411733,1.130115
597,-1.372089,-0.737806,0.528362,-0.392601,-0.395877,-0.554487,-0.683203,0.273231,-2.428760,1.130115
813,0.728816,2.236021,0.528362,-0.392601,-0.017890,0.670276,0.822993,0.273231,0.411733,1.130115
...,...,...,...,...,...,...,...,...,...,...
222,-1.372089,-0.737806,0.528362,-0.392601,-0.398498,0.400072,-0.017427,0.273231,0.411733,1.130115
721,0.728816,1.460556,0.528362,-0.392601,-0.201484,1.250100,1.560713,0.273231,-2.428760,-0.732058
271,0.728816,-0.737806,0.528362,-0.392601,0.940773,-0.554487,-0.112538,0.273231,0.411733,-1.250298
527,0.728816,0.253470,-1.892641,-0.392601,-0.019407,-0.065408,0.184684,0.273231,-2.428760,1.130115


In [140]:
y_test.value_counts()

1    190
0    190
Name: Loan_Status, dtype: int64

# Model training

In [141]:
log_reg = LogisticRegression(random_state=10)
log_reg.fit(x_train,y_train)

LogisticRegression(random_state=10)

# Evaluation

In [142]:
## Testing daat evaluation
y_pred_test = log_reg.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred_test)
print('Confusion matrix : \n', cnf_matrix)

multilabel_cnf_matrix = multilabel_confusion_matrix(y_test, y_pred_test)
print('Multilabel Confusion matrix : \n', multilabel_cnf_matrix)

acc_score_Logistic_Test = accuracy_score(y_test, y_pred_test)
print('Accuracy score : \n', acc_score_Logistic_Test)

class_report = classification_report(y_test, y_pred_test)
print('Classification report : \n', class_report)


Confusion matrix : 
 [[106  84]
 [ 16 174]]
Multilabel Confusion matrix : 
 [[[174  16]
  [ 84 106]]

 [[106  84]
  [ 16 174]]]
Accuracy score : 
 0.7368421052631579
Classification report : 
               precision    recall  f1-score   support

           0       0.87      0.56      0.68       190
           1       0.67      0.92      0.78       190

    accuracy                           0.74       380
   macro avg       0.77      0.74      0.73       380
weighted avg       0.77      0.74      0.73       380



In [143]:
# Training Data Evalutaion

y_pred_train = log_reg.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred_train)
print('Confusuion matrix :\n', cnf_matrix)

acc_score__Logistic_Train = accuracy_score(y_train,y_pred_train)
print('Accuracy score :\n', acc_score__Logistic_Train)

clf_report = classification_report(y_train,y_pred_train)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[122 110]
 [ 31 201]]
Accuracy score :
 0.6961206896551724
Classification report :
               precision    recall  f1-score   support

           0       0.80      0.53      0.63       232
           1       0.65      0.87      0.74       232

    accuracy                           0.70       464
   macro avg       0.72      0.70      0.69       464
weighted avg       0.72      0.70      0.69       464



# BY KNN

# Model training

In [144]:
knn_clf = KNeighborsClassifier(n_neighbors=5)  
knn_clf.fit(x_train,y_train)  # model saves training data

KNeighborsClassifier()

In [145]:
# Testing Data Evalutaion

y_pred = knn_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Test = accuracy_score(y_test,y_pred)
print('Accuracy score :\n', acc_score_KNN_Test)

clf_report = classification_report(y_test,y_pred)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[153  37]
 [ 54 136]]
Accuracy score :
 0.7605263157894737
Classification report :
               precision    recall  f1-score   support

           0       0.74      0.81      0.77       190
           1       0.79      0.72      0.75       190

    accuracy                           0.76       380
   macro avg       0.76      0.76      0.76       380
weighted avg       0.76      0.76      0.76       380



In [146]:
# Training Data Evalutaion

y_pred_train = knn_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred_train)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Train = accuracy_score(y_train,y_pred_train)
print('Accuracy score :\n', acc_score_KNN_Train)

clf_report = classification_report(y_train,y_pred_train)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[192  40]
 [ 46 186]]
Accuracy score :
 0.8146551724137931
Classification report :
               precision    recall  f1-score   support

           0       0.81      0.83      0.82       232
           1       0.82      0.80      0.81       232

    accuracy                           0.81       464
   macro avg       0.81      0.81      0.81       464
weighted avg       0.81      0.81      0.81       464



# KNN

In [147]:
knn_clf_norm = KNeighborsClassifier(n_neighbors=5)
knn_clf_norm.fit(x_train,y_train)

KNeighborsClassifier()

In [148]:
# Testing Data Evalutaion

y_pred = knn_clf_norm.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Test = accuracy_score(y_test,y_pred)
print('Accuracy score :\n', acc_score_KNN_Test)

clf_report = classification_report(y_test,y_pred)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[153  37]
 [ 54 136]]
Accuracy score :
 0.7605263157894737
Classification report :
               precision    recall  f1-score   support

           0       0.74      0.81      0.77       190
           1       0.79      0.72      0.75       190

    accuracy                           0.76       380
   macro avg       0.76      0.76      0.76       380
weighted avg       0.76      0.76      0.76       380



In [149]:
# Training Data Evalutaion

y_pred_train = knn_clf_norm.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred_train)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Train = accuracy_score(y_train,y_pred_train)
print('Accuracy score :\n', acc_score_KNN_Train)

clf_report = classification_report(y_train,y_pred_train)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[192  40]
 [ 46 186]]
Accuracy score :
 0.8146551724137931
Classification report :
               precision    recall  f1-score   support

           0       0.81      0.83      0.82       232
           1       0.82      0.80      0.81       232

    accuracy                           0.81       464
   macro avg       0.81      0.81      0.81       464
weighted avg       0.81      0.81      0.81       464



# Decision Tree classification

In [165]:
x_train, x_test, y_train, y_test = train_test_split(x_samples, y_samples, test_size = 0.45, random_state=11, stratify=y_samples)

In [168]:
dt_clf = DecisionTreeClassifier(random_state=10)
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier(random_state=10)

# Model Evaluation

In [169]:
# Testing Accuracy

y_pred = dt_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Test = accuracy_score(y_test, y_pred)
print("Accuracy Score : \n", acc_score_DT_Test)
print("********************************************")

clf_report = classification_report(y_test, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[152  38]
 [ 57 133]]
********************************************
Accuracy Score : 
 0.75
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.73      0.80      0.76       190
           1       0.78      0.70      0.74       190

    accuracy                           0.75       380
   macro avg       0.75      0.75      0.75       380
weighted avg       0.75      0.75      0.75       380

********************************************


In [170]:
# Training Accuracy

y_pred = dt_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Train = accuracy_score(y_train, y_pred)
print("Accuracy Score : \n", acc_score_DT_Train)
print("********************************************")

clf_report = classification_report(y_train, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[232   0]
 [  0 232]]
********************************************
Accuracy Score : 
 1.0
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       232
           1       1.00      1.00      1.00       232

    accuracy                           1.00       464
   macro avg       1.00      1.00      1.00       464
weighted avg       1.00      1.00      1.00       464

********************************************


# Plot DT

In [171]:
plt.figure(figsize = (200,160))
plot_tree(dt_clf, feature_names=x.columns, class_names = ["0", "1"], filled = True)
plt.savefig('DecisionTree_loan_pred.png')

# Hyperparameter Testing

In [172]:
dt_clf = DecisionTreeClassifier()
hyperparameters = {"criterion" : ["gini", "entropy"],
                  "max_depth" : np.arange(3,8),
                  "min_samples_split" : np.arange(2,20),
                  "min_samples_leaf" : np.arange(2,10)}

In [173]:
rscv_dt_model = RandomizedSearchCV(dt_clf, hyperparameters, cv = 5)
rscv_dt_model.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([3, 4, 5, 6, 7]),
                                        'min_samples_leaf': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])})

In [174]:
rscv_dt_model.best_estimator_

DecisionTreeClassifier(max_depth=5, min_samples_leaf=6, min_samples_split=11)

In [177]:
dt_clf = DecisionTreeClassifier(max_depth=3, min_samples_leaf=8, min_samples_split=13)
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=8, min_samples_split=13)

In [178]:
# Testing Accuracy

y_pred = dt_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Test = accuracy_score(y_test, y_pred)
print("Accuracy Score : \n", acc_score_DT_Test)
print("********************************************")

clf_report = classification_report(y_test, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[140  50]
 [ 56 134]]
********************************************
Accuracy Score : 
 0.7210526315789474
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.71      0.74      0.73       190
           1       0.73      0.71      0.72       190

    accuracy                           0.72       380
   macro avg       0.72      0.72      0.72       380
weighted avg       0.72      0.72      0.72       380

********************************************


In [179]:
# Training Accuracy

y_pred = dt_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Train = accuracy_score(y_train, y_pred)
print("Accuracy Score : \n", acc_score_DT_Train)
print("********************************************")

clf_report = classification_report(y_train, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[162  70]
 [ 59 173]]
********************************************
Accuracy Score : 
 0.7219827586206896
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.73      0.70      0.72       232
           1       0.71      0.75      0.73       232

    accuracy                           0.72       464
   macro avg       0.72      0.72      0.72       464
weighted avg       0.72      0.72      0.72       464

********************************************


# Random Forest

In [180]:
rf_clf = RandomForestClassifier(random_state=10)
rf_clf.fit(x_train, y_train)

RandomForestClassifier(random_state=10)

In [182]:
# Testing Data Evaluation
y_pred = rf_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion Matrix \n", cnf_matrix)

acc_score_RF_Test = accuracy_score(y_test,y_pred)
print("Accuracy score \n", acc_score_RF_Test)

clf_report = classification_report(y_test,y_pred)
print("Classification report \n", clf_report)

Confusion Matrix 
 [[156  34]
 [ 33 157]]
Accuracy score 
 0.8236842105263158
Classification report 
               precision    recall  f1-score   support

           0       0.83      0.82      0.82       190
           1       0.82      0.83      0.82       190

    accuracy                           0.82       380
   macro avg       0.82      0.82      0.82       380
weighted avg       0.82      0.82      0.82       380



In [183]:
# Training Data Evaluation
y_pred = rf_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred)
print("Confusion Matrix \n", cnf_matrix)

acc_score_RF_Train = accuracy_score(y_train,y_pred)
print("Accuracy score \n", acc_score_RF_Train)

clf_report = classification_report(y_train,y_pred)
print("Classification report \n", clf_report)

Confusion Matrix 
 [[232   0]
 [  0 232]]
Accuracy score 
 1.0
Classification report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       232
           1       1.00      1.00      1.00       232

    accuracy                           1.00       464
   macro avg       1.00      1.00      1.00       464
weighted avg       1.00      1.00      1.00       464



# Hyperparameter Tuning

In [184]:
rf_model = RandomForestClassifier(random_state=10)

hyp = {"n_estimators" : np.arange(10,200),
    "criterion":['gini','entropy'],
    "max_depth":np.arange(5,15),
    "min_samples_split":np.arange(5,20),
    "min_samples_leaf":np.arange(5,15),
    "max_features":['auto']}

In [185]:
rscv_rf_clf =RandomizedSearchCV(rf_model, hyp, cv = 5)
rscv_rf_clf.fit(x_train,y_train)
rscv_rf_clf.best_estimator_

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

RandomForestClassifier(criterion='entropy', max_depth=9, max_features='auto',
                       min_samples_leaf=5, min_samples_split=14,
                       n_estimators=25, random_state=10)

In [186]:
rf_clf = RandomForestClassifier(criterion='entropy', max_depth=9, min_samples_leaf=5,
                       min_samples_split=14, n_estimators=25, random_state=10)
rf_clf.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=9, min_samples_leaf=5,
                       min_samples_split=14, n_estimators=25, random_state=10)

In [187]:
# Testing Data Evaluation
y_pred = rf_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion Matrix", cnf_matrix)

acc_score_RF_Test = accuracy_score(y_test,y_pred)
print("Accuracy score", acc_score_RF_Test)

clf_report = classification_report(y_test,y_pred)
print("Classification report", clf_report)

Confusion Matrix [[142  48]
 [ 35 155]]
Accuracy score 0.781578947368421
Classification report               precision    recall  f1-score   support

           0       0.80      0.75      0.77       190
           1       0.76      0.82      0.79       190

    accuracy                           0.78       380
   macro avg       0.78      0.78      0.78       380
weighted avg       0.78      0.78      0.78       380



In [188]:
# Training Data Evaluation
y_pred = rf_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred)
print("Confusion Matrix", cnf_matrix)

acc_score_RF_Train = accuracy_score(y_train,y_pred)
print("Accuracy score", acc_score_RF_Train)

clf_report = classification_report(y_train,y_pred)
print("Classification report", clf_report)

Confusion Matrix [[191  41]
 [ 21 211]]
Accuracy score 0.8663793103448276
Classification report               precision    recall  f1-score   support

           0       0.90      0.82      0.86       232
           1       0.84      0.91      0.87       232

    accuracy                           0.87       464
   macro avg       0.87      0.87      0.87       464
weighted avg       0.87      0.87      0.87       464



In [190]:
df_loan = pd.DataFrame(columns=['train_acc','Test_acc'],index=['Logistic_Loan','KNN_Loan','DT_Loan','RF_Loan'])
df_loan

,train_acc,Test_acc
Logistic_Loan,NaN,NaN
KNN_Loan,NaN,NaN
DT_Loan,NaN,NaN
RF_Loan,NaN,NaN


In [195]:
df_loan['train_acc'] = [acc_score__Logistic_Train,acc_score_KNN_Train,acc_score_DT_Train,acc_score_RF_Train]
df_loan['Test_acc'] = [acc_score_Logistic_Test,acc_score_KNN_Test,acc_score_DT_Test,acc_score_RF_Test]

In [196]:
df_loan.to_csv('df_loan.csv')
df_loan

,train_acc,Test_acc
Logistic_Loan,0.696121,0.736842
KNN_Loan,0.814655,0.760526
DT_Loan,0.721983,0.721053
RF_Loan,0.866379,0.781579
